In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
from sklearn.preprocessing import normalize
from joblib import Parallel, delayed
from scipy import linalg

class PGMHQC_fast(BaseEstimator, ClassifierMixin):
    """The Pretty Good Measurement (PGM) - Helstrom Quantum Centroid (HQC) classifier is a 
    quantum-inspired supervised classification approach for data with multiple classes.
                         
    Parameters
    ----------
    rescale : int or float, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    encoding : str, default = 'amplit'
        The encoding method used to encode vectors into quantum densities. Possible values:
        'amplit', 'stereo'. 'amplit' means using the amplitude encoding method. 'stereo' means 
        using the inverse of the standard stereographic projection encoding method. Default set 
        to 'amplit'.
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking 
        the n-fold Kronecker tensor product for each quantum density.
    measure : str, default = 'pgm'
        The measurement used to distinguish between quantum states. Possible values: 'pgm', 
        'hels'. The value 'pgm' stands for "Pretty Good Measurement", 'hels' stands for 
        "Helstrom measurement" (applicable only for binary classification). Default set to 
        'pgm'.
    class_weight : str, default = None 
        Weights associated with classes. This is the class weights assigned to the quantum 
        centroids in the Pretty Good Measurement or Helstrom observable. Possible values: None,
        'balanced'. If None given, all classes are supposed to have weight one. The 'balanced' 
        mode uses the values of y to automatically adjust weights inversely proportional to class
        frequencies in the input data as n_samples / (n_classes * np.bincount(y)). Default set
        to None.
    n_jobs : int, default = None
        The number of CPU cores used when parallelizing. If -1 all CPUs are used. If 1 is given, 
        no parallel computing code is used at all. For n_jobs below -1, (n_cpus + 1 + n_jobs) 
        are used. Thus for n_jobs = -2, all CPUs but one are used. None is a marker for ‘unset’ 
        that will be interpreted as n_jobs = 1.
    n_splits : int, default = 1
        The number of subset splits performed on the input dataset row-wise and on the number 
        of eigenvalues/eigenvectors of the Quantum Helstrom observable for optimal speed 
        performance. If 1 is given, no splits are performed. For optimal speed, recommend using 
        n_splits = int(numpy.ceil(number of CPU cores used/number of classes)). If memory blow-out 
        occurs, reduce n_splits. When n_splits = 1 and memory blow-out still occurs, reduce n_jobs.
    
    Attributes
    ----------
    classes_ : ndarray, shape (n_classes,)
        Sorted classes.
    qcentroids_ : ndarray, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Quantum Centroids for each class.
    pgms_ : list, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Values for the Pretty Good Measurements. Only applicable when Pretty Good Measurement is 
        selected.
    pgm_bound_ : float
        Pretty Good Measurement bound is the upper bound on the probability that one can correctly
        discriminate whether a quantum density is of which of the (multiclass) N quantum density 
        patterns. Only applicable when Pretty Good Measurement is selected.
    proj_sums_ : list, shape (n_classes, (n_features + 1)**n_copies, (n_features + 1)**n_copies)
        Sum of the projectors of the Quantum Helstrom observable's unit eigenvectors, which has
        corresponding positive and negative eigenvalues respectively. Only applicable when Helstrom
        Measurement is selected.
    hels_bound_ : float
        Helstrom bound is the upper bound on the probability that one can correctly 
        discriminate whether a quantum density is of which of the two binary quantum density 
        patterns. Only applicable when Helstrom Measurement is selected.         
    """
    # Initialize model hyperparameters
    def __init__(self, 
                 rescale = 1,
                 encoding = 'amplit',
                 n_copies = 1, 
                 measure = 'pgm',
                 class_weight = None, 
                 n_jobs = None, 
                 n_splits = 1):
        self.rescale = rescale
        self.encoding = encoding
        self.n_copies = n_copies
        self.measure = measure
        self.class_weight = class_weight
        self.n_jobs = n_jobs
        self.n_splits = n_splits
        

    # Function for X_prime, set as global function
    global X_prime_func
    def X_prime_func(self, X, m):
        # Cast X to float to ensure all following calculations below are done in float
        # rather than integer
        X = X.astype(float)
        
        # Rescale X
        X = self.rescale*X
        
        # Calculate sum of squares of each row (sample) in X
        X_sq_sum = (X**2).sum(axis = 1)
        
        # Calculate X' using amplitude or inverse of the standard stereographic projection
        # encoding method
        if self.encoding == 'amplit':
            X_prime = normalize(np.concatenate((X, np.ones(m).reshape(-1, 1)), axis = 1))
        elif self.encoding == 'stereo':
            X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1) \
                      *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1))
        else:
            raise ValueError('encoding should be "amplit" or "stereo"')
        return X_prime  
    
        
    # Function for kronecker tensor product with broadcasting, set as global function
    global kronecker
    def kronecker(A, B):
        return np.einsum('nab,ncd->nacbd', A, B).reshape(A.shape[0],
                                                         A.shape[1]*B.shape[1],
                                                         A.shape[2]*B.shape[2])
    

    # Set np.einsum subscripts (between unnested and nested objects) as a constant, set as global 
    # variable
    global einsum_unnest, einsum_nest
    einsum_unnest = 'ij,ji->'
    einsum_nest = 'bij,ji->b'
    
    
    # Function for fit
    def fit(self, X, y):
        """Perform PGM-HQC classification with the amplitude and inverse of the standard 
        stereographic projection encoding methods, with the option to rescale the dataset prior 
        to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Check data in X and y as required by scikit-learn v0.25
        X, y = self._validate_data(X, y, reset = True)
        
        # Ensure target y is of non-regression type  
        # Added as required by sklearn check_estimator
        check_classification_targets(y)
    
        # Store classes and encode y into class indexes
        self.classes_, y_class_index = np.unique(y, return_inverse = True)
                
        # Number of classes, set as global variable
        global num_classes
        num_classes = len(self.classes_)
        
        # Raise error when there are more than 2 classes and Helstrom measurement is specified
        if num_classes > 2 and self.measure == 'hels':
            raise ValueError('Helstrom measurement can be applied for binary classification only')
        else:
            # Number of rows and columns in X
            m, n = X.shape[0], X.shape[1]
            
            # Calculate X_prime
            X_prime = X_prime_func(self, X, m)
        
            # Number of columns in X', set as global variable
            global n_prime
            n_prime = n + 1
        
            # Function to calculate number of rows (samples) and Quantum Centroids for each class
            def qcentroids_terms_func(i):
                # Determine rows (samples) in X' belonging to each class
                X_prime_class = X_prime[y_class_index == i]
            
                # Number of rows (samples) in X' belonging to either class
                m_class = X_prime_class.shape[0]
            
                # Split X' belonging to each class into n_splits subsets, row-wise
                X_prime_class_split = np.array_split(X_prime_class,
                                                     indices_or_sections = self.n_splits,
                                                     axis = 0)
            
                # Function to calculate the Quantum Centroids for each class, per subset split
                def X_prime_class_split_func(j):
                    # Counter for j-th split of X'
                    X_prime_class_split_jth = X_prime_class_split[j]
                
                    # Number of rows (samples) in j-th split of X'
                    m_class_split = X_prime_class_split_jth.shape[0]
                
                    # Encode vectors into quantum densities
                    density_split = np.matmul(X_prime_class_split_jth.reshape(m_class_split, 
                                                                              n_prime, 1),
                                              X_prime_class_split_jth.reshape(m_class_split, 
                                                                              1, n_prime))
                                        
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:
                        density_split = density_split
                    else:
                        density_split_copy = density_split
                        for _ in range(self.n_copies - 1):
                            density_split = kronecker(density_split, density_split_copy)
                
                    # Calculate sum of quantum densities
                    density_split_sum = density_split.sum(axis = 0)
                
                    # Calculate Quantum Centroid for each class, per subset split
                    # Added ZeroDivisionError as required by scikit-learn check_estimator()
                    try:
                        qcentroid = (1/m_class)*density_split_sum
                    except ZeroDivisionError:
                        qcentroid = 0
                    return qcentroid
                return m_class, np.sum(Parallel(n_jobs = self.n_jobs)(delayed(X_prime_class_split_func)(j) 
                                                                      for j in range(self.n_splits)), axis = 0)
        
            # Calculate number of rows (samples) and Quantum Centroids for each class
            # Added dtype = object as required by NumPy v19.0 when creating ndarray from ragged nested sequences
            qcentroids_terms = np.array(Parallel(n_jobs = self.n_jobs)(delayed(qcentroids_terms_func)(i) 
                                                                       for i in range(num_classes)), dtype = object)

            # Determine Quantum Centroids
            self.qcentroids_ = qcentroids_terms[:, 1]
            
            # Calculate class weight
            if self.class_weight == None:
                class_weight_terms = qcentroids_terms[:, 0]/m
            elif self.class_weight == 'balanced':
                class_weight_terms = np.array([1/num_classes for k in range(num_classes)])
            else:
                raise ValueError('class_weight should be None or "balanced"')
                
            # When Pretty Good Measurement is specified
            if self.measure == 'pgm':                    
                # Function to calculate R
                def R_func(a):
                    return class_weight_terms[a]*self.qcentroids_[a]
                
                # Calculate R
                R = np.sum(Parallel(n_jobs = self.n_jobs)(delayed(R_func)(a) for a in range(num_classes)), axis = 0)
                
                # Calculate square root of the pseudo inverse of R, and remove complex part of the matrix
                # created due to numerical precision/rounding issues in machine language
                sqrt_pinv_R = np.real(linalg.sqrtm(np.linalg.pinv(R, hermitian = True)))
                    
                # Calculate kernel of R
                ker_R = linalg.null_space(R)
                    
                # Calculate projector of kernel of R
                proj_ker_R = np.dot(ker_R, ker_R.T)
                    
                # Function to calculate Pretty Good Measurement
                def pgm_func(b):
                    return np.linalg.multi_dot([sqrt_pinv_R, class_weight_terms[b]*self.qcentroids_[b], \
                                                sqrt_pinv_R]) + (1/num_classes)*proj_ker_R
            
                # Calculate Pretty Good Measurement
                self.pgms_ = Parallel(n_jobs = self.n_jobs)(delayed(pgm_func)(b) for b in range(num_classes))
                
                # Function to calculate PGM bound
                def pgm_bound_func(c):
                    return class_weight_terms[c]*np.einsum(einsum_unnest, self.qcentroids_[c], self.pgms_[c])
                
                # Calculate PGM bound
                self.pgm_bound_ = np.sum(Parallel(n_jobs = self.n_jobs)(delayed(pgm_bound_func)(c) 
                                                                        for c in range(num_classes)), axis = 0)
            # When Helstrom measurement is specified
            elif self.measure == 'hels':               
                # Calculate quantum Helstrom observable
                hels_obs = class_weight_terms[0]*self.qcentroids_[0] \
                           - class_weight_terms[1]*self.qcentroids_[1]
            
                # Number of rows/columns in density matrix, set as global variable
                global density_nrow_ncol
                density_nrow_ncol = hels_obs.shape[0]
            
                # Calculate eigenvalues w and unit eigenvectors v of the quantum Helstrom observable
                w, v = np.linalg.eigh(hels_obs)
                
                # Length of w
                len_w = len(w)
                
                # Initialize array eigval_class
                eigval_class = np.empty_like(w)
                for b in range(len_w):
                    # Create an array of 0s and 1s to indicate positive and negative eigenvalues
                    # respectively
                    if w[b] > 0:
                        eigval_class[b] = 0
                    else:
                        eigval_class[b] = 1
                        
                # Transpose matrix v containing unit eigenvectors to row-wise
                eigvec = v.T
                
                # Function to calculate sum of the projectors corresponding to positive and negative
                # eigenvalues respectively
                def sum_proj_func(c):
                    # Determine unit eigenvectors belonging to positive and negative eigenvalues 
                    # respectively
                    eigvec_class = eigvec[eigval_class == c]
                    
                    # Split unit eigenvectors into n_splits subsets
                    eigvec_class_split = np.array_split(eigvec_class,
                                                        indices_or_sections = self.n_splits,
                                                        axis = 0)
                    
                    # Function to calculate sum of the projectors corresponding to positive and negative
                    # eigenvalues respectively, per subset split
                    def eigvec_class_split_func(d):
                        # Counter for d-th split of eigvec_class_split
                        eigvec_class_split_dth = eigvec_class_split[d]
                    
                        # Number of rows (samples) in d-th split of eigvec_class_split
                        m_eigvec_class_split = eigvec_class_split_dth.shape[0]
                    
                        # Calculate projectors corresponding to positive and negative eigenvalues
                        # respectively, per subset split
                        proj_split = np.matmul(eigvec_class_split_dth.reshape(m_eigvec_class_split,
                                                                              density_nrow_ncol, 1),
                                               eigvec_class_split_dth.reshape(m_eigvec_class_split,
                                                                              1, density_nrow_ncol))

                        # Calculate sum of projectors
                        proj_split_sum = proj_split.sum(axis = 0)
                        return proj_split_sum
                    return np.sum(Parallel(n_jobs = self.n_jobs)(delayed(eigvec_class_split_func)(d) 
                                                                 for d in range(self.n_splits)), axis = 0)
                
                # Calculate sum of the projectors corresponding to positive and negative eigenvalues
                # respectively
                self.proj_sums_ = Parallel(n_jobs = self.n_jobs)(delayed(sum_proj_func)(c) for c in range(2))
                
                # Function to calculate Helstrom bound
                def hels_bound_func(e):
                    return class_weight_terms[e]*np.einsum(einsum_unnest, self.qcentroids_[e], self.proj_sums_[e])
                
                # Calculate Helstrom bound
                self.hels_bound_ = np.sum(Parallel(n_jobs = self.n_jobs)(delayed(hels_bound_func)(e) \
                                                                         for e in range(2)))
            # When Pretty Good Measurement or Helstrom measurement is misspecified
            else:
                raise ValueError('measure should be "pgm" or "hels"')
        return self             

    
    # Function for predict_proba
    def predict_proba(self, X):
        """Performs PMG-HQC classification on X and returns the trace of the dot product of the 
        densities and the POV (positive operator-valued) measure, i.e. the class probabilities.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : ndarray, shape (n_samples, n_classes)
            Each column corresponds to the trace of the dot product of the densities and the POV 
            (positive operator-valued) measure for each class, i.e. each column corresponds to the 
            class probabilities. An array of float.
        """
        # Check if fit had been called
        if self.measure == 'pgm':
            check_is_fitted(self, ['pgms_'])
        else:
            check_is_fitted(self, ['proj_sums_'])

        # Check data in X as required by scikit-learn v0.25
        X = self._validate_data(X, reset = False)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Calculate X_prime
        X_prime = X_prime_func(self, X, m)
               
        # Function to calculate trace values for each class
        def trace_func(i):
            # Split X' into n_splits subsets, row-wise
            X_prime_split = np.array_split(X_prime, 
                                           indices_or_sections = self.n_splits, 
                                           axis = 0)
            
            # Function to calculate trace values for each class, per subset split
            def trace_split_func(j):
                # Counter for j-th split X'
                X_prime_split_jth = X_prime_split[j]
                
                # Number of rows (samples) in j-th split X'
                X_prime_split_m = X_prime_split_jth.shape[0]
                
                # Encode vectors into quantum densities
                density_chunk = np.matmul(X_prime_split_jth.reshape(X_prime_split_m, n_prime, 1),
                                          X_prime_split_jth.reshape(X_prime_split_m, 1, n_prime))
                
                # Calculate n-fold Kronecker tensor product
                if self.n_copies == 1:
                    density_chunk = density_chunk
                else:
                    density_chunk_copy = density_chunk
                    for _ in range(self.n_copies - 1):
                        density_chunk = kronecker(density_chunk, density_chunk_copy)
                        
                # When Pretty Good Measurement is specified
                if self.measure == 'pgm':
                    # Calculate trace of the dot product of density of each row and Pretty Good 
                    # Measurement
                    trace_class_split = np.einsum(einsum_nest, density_chunk, self.pgms_[i])
                # When Helstrom measurement is specified
                else:               
                    # Calculate trace of the dot product of density of each row and sum of 
                    # projectors with corresponding positive and negative eigenvalues respectively
                    trace_class_split = np.einsum(einsum_nest, density_chunk, self.proj_sums_[i])
                return trace_class_split
            
            # Calculate trace values for each class, per subset split
            trace_class = Parallel(n_jobs = self.n_jobs)(delayed(trace_split_func)(j) 
                                                         for j in range(self.n_splits))
            return np.concatenate(trace_class, axis = 0)
        
        # Calculate trace values for each class
        trace_matrix = np.transpose(Parallel(n_jobs = self.n_jobs)(delayed(trace_func)(i) 
                                                                   for i in range(num_classes)))
        return trace_matrix
        
    
    # Function for predict
    def predict(self, X):
        """Performs PGM-HQC classification on X and returns the classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : ndarray, shape (n_samples,)
            The predicted classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If columns have the same trace value, returns column with the smallest column index value
        predict_trace_index = np.argmax(self.predict_proba(X), axis = 1)
        # Returns the predicted classes
        return self.classes_[predict_trace_index]

In [2]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(PGMHQC_fast())

ValueError: class_weight should be None or "balanced"